## Differentialoperators

To learn a solution of a differential equation, one needs to compute different derivatives of the neural network. 

To make the implementation of a given ODE/PDE easier, different operators are already implemented. They can be found under
torchphysics.utils.differentialoperators.

#### Implemented Operators:
* For scalar outputs: 
    * _grad_, to compute the gradient $\nabla u$
    * _laplacian_, to compute the laplace operator $\Delta u$ 
    * _partial_, to compute a partial derivative $\partial_x u$ 
    * _normal_derivatives_, to compute the normal derivative $\vec{n} \cdot \nabla u$
* For vector outputs:
    * _div_, to compute the divergence $\text{div}(u)$ or $\nabla \cdot u$
    * _rot_, to compute the rotation/curl of a vector field $\nabla \times u$
    * _jac_, to compute the jacobian matrix 

Of course, the operators for scalar outputs can also be used for vectorial outputs, if one output entry is specified. E.g. $u: D \to \mathbb{R}^3$ then do $\text{laplacian}(u[:, 1], x)$, to get the laplacian of the first entry.

All operators can handle the computation on a whole data batch.

To better understand the usage of these operators, it follows a small example. Under the hood, all operators use the autograd functionallity of PyTorch. Therefore we need to work with tensor. To test the operators, we use the function $f: \mathbb{R}^2 \times \mathbb{R} \to \mathbb{R}, f(x, t) = \sin(t  x_1) + x_2^2$.

In [1]:
import torch
# Define the function:
def f(x, t):
    return torch.sin(t * x[:, :1]) + x[:, 1:]**2

# Define some points where to evaluate the function
x = torch.tensor([[1.0, 1.0], [0, 1], [1, 0]], requires_grad=True) 
t = torch.tensor([[1], [0], [2.0]], requires_grad=True)
# requires_grad=True is needed, so PyTorch knows to create a backwards graph.
# These tensors could be seen as a batch with three data points.

Another important part of the implemented operators is, that the input is not a function f, but the already computed outputs f(x, t). The 
advantage is, that when many different operators are used, the forward pass only happens once.
If a custom operator is implemented, the operator also needs to work like this.  

In [2]:
# Therefore comput now the outputs:
out = f(x, t)

Let's compute the gradient and laplacian of our function, w.r.t. $x$. Analytically they are:
$$
    \nabla f(x, t) = (t\cos(tx_1), 2x_2)
$$
$$
    \Delta f(x, t) = -t^2\sin(tx_1) + 2
$$


In [8]:
from torchphysics.utils import grad, laplacian
print('Gradient w.r.t. x:')
print(grad(out, x))
print('Laplace w.r.t. x:')
print(laplacian(out, x))

Gradient w.r.t. x:
tensor([[ 0.5403,  2.0000],
        [ 0.0000,  2.0000],
        [-0.8323,  0.0000]], grad_fn=<AddBackward0>)
Laplace w.r.t. x:
tensor([[ 1.1585],
        [ 2.0000],
        [-1.6372]], grad_fn=<AddBackward0>)


One could now check that these values are correct, by inputting the $x$ and $t$ values in the analytic expression.

It is also possible to create the gradient/laplacian with respect to $t$. Since $t$ is one dimensional this, would be just the first/second derivative: 


In [6]:
print('Gradient w.r.t. t:')
print(grad(out, t))
print('Laplace w.r.t. t:')
print(laplacian(out, t))

Gradient w.r.t. t:
tensor([[ 0.5403],
        [ 0.0000],
        [-0.4161]], grad_fn=<MulBackward0>)
Laplace w.r.t. t:
tensor([[-0.8415],
        [ 0.0000],
        [-0.9093]], grad_fn=<AddBackward0>)


These are the basic on how to use the implemented differential operators. The next step would be creating a condition that the neural network has to fulfill. This is shown in _create_condition.ipynb_.